In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../')
print(os.getcwd())
from v2.model import Trading
import plotly.graph_objs as go
from plotly.offline import plot
from scipy.signal import argrelextrema



/mnt/c/Users/jam60/OneDrive/Repos/vivaldi/back_testing


In [2]:


def load_config():
    my_config = {}
    with open('config.config') as config:
        for line in config:
            args = line.split('=')
            my_config[args[0]] = args[1].rstrip().split(',')
    return my_config

model = Trading(load_config())



In [3]:
#this only works on one dataset at a time
candle, buys, sells = None, None, None
datasets = model.dfs
fee = 0.0026
for d in datasets:
    buy_prices = []
    buy_times = []
    sell_prices = []
    sell_times = []
    name = d[1]
    cur_min = 99999999999999999.00
    cur_max = 0.00
    cur_min_time = 00.0
    cur_max_time = 00.0
    last_buy_price = 000.00
    last_buy_time = 000.00
    sell_time = 0000.0
    dataset = d[0]
    looking_for_buy = True
    for row in dataset.itertuples():
        close = row.close
        time = row.time
        if looking_for_buy:
            if close < cur_min:
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time
            elif close > cur_max:
                cur_max = close
                cur_max_time = time
                delta = cur_max - cur_min
                if delta > ((cur_min * fee) + (cur_max * fee)):
                    looking_for_buy = False
                    last_buy_price = cur_min
                    last_buy_time = cur_min_time
        else:
            trade_fee = ((cur_min * fee) + (cur_max * fee))
            if close > cur_max:
                cur_max = close
                cur_max_time = time
            elif close < cur_max - trade_fee:
                looking_for_buy = True
                buy_prices.append(cur_min)
                buy_times.append(cur_min_time)
                sell_prices.append(cur_max)
                sell_times.append(cur_max_time)
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time

    candle = go.Candlestick(x=dataset['time'], open=dataset['open'], close=dataset['close'], high=dataset['high'], low=dataset['low'], name='Candlesticks')

    buys = go.Scatter(x=buy_times, y=buy_prices, name='Optimal Entries', mode='markers')
    sells = go.Scatter(x=sell_times, y=sell_prices, name='Optimal Exits', mode='markers')
    dataset['trough'] = dataset.iloc[argrelextrema(dataset.close.values, np.less_equal, order=480)[0]]['close']
    dataset['trough'] = dataset['trough'].fillna(0).gt(0).astype(int)
    troughs = go.Scatter(x=dataset['time'], y=dataset['trough'], name='Troughs (Local Maximums)', mode='markers')
    data = [buys] + [sells] + [candle] + [troughs]
    layout = go.Layout(title='Optimal Buying Times/Detected Troughs for ' + name)
    fig = go.Figure(data=data, layout=layout)
    plot(fig, filename='plots/optimal_' + name + '.html')







In [4]:
sell_prices

[9588.4,
 9609.5,
 10195.0,
 10119.0,
 10355.3,
 10229.0,
 10206.2,
 10128.8,
 10146.0,
 10212.3,
 9768.6,
 9596.2,
 9605.0,
 9569.4,
 9499.6,
 9544.1,
 9548.0,
 9530.0,
 9634.7,
 9594.6,
 9616.1,
 9690.0,
 9684.0,
 9781.2,
 9840.9,
 9883.8,
 9818.6,
 9855.1,
 9841.9,
 9801.6,
 9850.0,
 9733.8,
 9690.0,
 9725.9,
 9718.3,
 9749.2,
 9607.2,
 9636.4,
 9731.0,
 9726.8,
 9696.9,
 9714.6,
 9670.1,
 9596.3,
 9786.9,
 9785.0,
 9787.0,
 9745.5,
 9880.0,
 9720.9,
 9748.6,
 9831.5,
 9814.8,
 9989.0,
 9900.0,
 9960.0,
 9832.9,
 9660.8,
 9564.9,
 9543.9,
 9381.5,
 9339.9,
 9366.9,
 9299.6,
 9505.9,
 9544.9,
 9449.0,
 9436.9,
 9479.4,
 9465.1,
 9466.0,
 9486.5,
 9474.9,
 9439.0,
 9431.6,
 9394.0,
 9274.9,
 9212.0,
 9062.2,
 9059.9,
 9144.1,
 9127.1,
 9395.0,
 9381.5,
 9448.4,
 9496.3,
 9584.7,
 9542.9,
 9583.0,
 9550.4,
 9508.9,
 9534.6,
 9520.5,
 9547.5,
 9420.2,
 9420.0,
 9461.5,
 9473.3,
 9451.2,
 9430.2,
 9410.0,
 9400.1,
 9318.4,
 9400.9,
 9421.1,
 9334.1,
 9342.0,
 9340.0,
 9322.9,
 9407.1,
 9

In [5]:
len(buy_prices)

147